In [ ]:
%pylab inline

In [ ]:
from __future__ import division 

import os
import glob
import ConfigParser

import numpy as np
import h5py
import pylab as plt

import cv2
import astra

from ipywidgets import interact
from pprint import pprint

In [ ]:
data_root = '/diskmnt/a/makov/yaivan/MMC_1/'
raw_root = os.path.join(data_root,'Raw')
out_file = os.path.join(data_root, 'raw.h5')

In [ ]:
# files = !ls {raw_root}
# pprint(sorted(files))

In [ ]:
def print_config(config):
    for k,v in config._sections.iteritems():
        print '[{}]'.format(k)
        for kk, vv in v.iteritems():
            print '\t{} = {}'.format(kk,vv)

config = ConfigParser.RawConfigParser()
config.optionxform = str
config.read(os.path.join(raw_root,'MMC1_2.82um_.log'))
print_config(config)

In [ ]:
object_name = config._sections['Acquisition']['Filename Prefix']
print 'Object name:', object_name

In [ ]:
data_files  = glob.glob(os.path.join(raw_root,object_name+'[0-9]'*4+'.tif'))
data_files = sorted(data_files)
print 'Data files found:', len(data_files)
print 'First file:', data_files[0]
print 'Last file:', data_files[-1]

In [ ]:
test_data = plt.imread(data_files[0])
print 'Size of image:', test_data.shape
print 'Image data type:', test_data.dtype

In [ ]:
# if not packed raw HDF5 file found, create it
#TODO: add adittional reference iif files
if not os.path.exists(out_file):
    with h5py.File(out_file,'w-') as h5f:
        h5f.create_dataset('data', (len(data_files), test_data.shape[0], test_data.shape[1]),
            chunks=True, dtype=test_data.dtype)
        for idf, df in enumerate(data_files):
            print idf,
            if idf%20 ==0: 
                print
            df = plt.imread(data_files[idf])
            h5f['data'][idf]=df        

In [ ]:
#opencv defined projector-backprojector
def cv_rotate(x, angle):
    """
    Rotate square array using OpenCV2 around center of the array
    :param x: 2d numpy array
    :param angle: angle in degrees
    :return: rotated array
    """
    x_center = tuple(
        np.array((x.shape[1], x.shape[0]), dtype='float32') / 2.0 - 0.5)
    rot_mat = cv2.getRotationMatrix2D(x_center, angle, 1.0)
    xro = cv2.warpAffine(
        x, rot_mat, (x.shape[1], x.shape[0]), flags=cv2.INTER_LINEAR)
    return xro


def cv_project(src):
    """
    Project 2D array to 1D array

    :param src: 2d numpy array.
    :return:
    """
    return np.squeeze(cv2.reduce(src, dim=1, rtype=cv2.cv.CV_REDUCE_SUM))


def cv_backproject(src):
    """
    Back project 1d array to 2d array.

    :param src: 1d numpy array
    :return:
    """
    ntimes = len(src)
    tmp_array = src / float(ntimes)
    return cv2.repeat(tmp_array, 1, ntimes)


def my_sart(sinogram, angles, input_coeff):
    """
    Clear SART tomography reconstruction. Without any tirks.

    :param sinogram: 2d array (s[:,i] - i-th projection)
    :param angles:  1d array of angles
    :param input_coeff:  
    :return:
    """
    reconst_shape = np.array(sinogram.shape[0], dtype='int32')
    tomo_rec = np.zeros((reconst_shape, reconst_shape), dtype='float32')
    tmp_backproj = np.zeros_like(tomo_rec)
    tmp_rot = np.zeros_like(tomo_rec)
    # set relaxation coeffitients here (one per iteration)
    for coeff in input_coeff:
        shuffle_iang = np.arange(len(angles))
        np.random.shuffle(shuffle_iang)
        for iang in shuffle_iang:
            tmp_rot = cv_rotate(tomo_rec, angles[iang])
            tmp_proj = sinogram[:, iang] - cv_project(tmp_rot)
            tmp_backproj = cv_backproject(tmp_proj * coeff)
            tmp_rot = cv_rotate(tmp_backproj, -angles[iang])
            tomo_rec += tmp_rot
        tomo_rec -= 0.5 * tomo_rec * (tomo_rec < 0)
    return tomo_rec

In [ ]:
def astra_tomo2d_parallel(sinogram, angles):
    angles = angles.astype('float64')
    detector_size = sinogram.shape[1]
    

    rec_size = detector_size
    vol_geom = astra.create_vol_geom(rec_size, rec_size)
    proj_geom = astra.create_proj_geom('parallel', 1.0, detector_size, angles)


    sinogram_id = astra.data2d.create('-sino', proj_geom, data=sinogram)
    # Create a data object for the reconstruction
    rec_id = astra.data2d.create('-vol', vol_geom)

    # Set up the parameters for a reconstruction algorithm using the GPU
    cfg = astra.astra_dict('SIRT_CUDA')
    cfg['ReconstructionDataId'] = rec_id
    cfg['ProjectionDataId'] = sinogram_id
    cfg['option'] = {}
    #cfg['option']['MinConstraint'] = 0
    # cfg['option']['MaxConstraint'] = 5

    # Available algorithms:
    # SIRT_CUDA, SART_CUDA, EM_CUDA, FBP_CUDA (see the FBP sample)

    # Create the algorithm object from the configuration structure
    alg_id = astra.algorithm.create(cfg)

    # Run 150 iterations of the algorithm
    astra.algorithm.run(alg_id, 150)

    # Get the result
    rec = astra.data2d.get(rec_id)

    # Clean up. Note that GPU memory is tied up in the algorithm object,
    # and main RAM in the data objects.
    astra.algorithm.delete(alg_id)
    astra.data2d.delete(rec_id)
    astra.data2d.delete(sinogram_id)
    astra.clear()
    return rec

In [ ]:
def astra_tomo2d_fanflat(sinogram, angles):
    angles = angles.astype('float64')
    detector_size = sinogram.shape[1]
    

    rec_size = detector_size
    vol_geom = astra.create_vol_geom(rec_size, rec_size)
#     proj_geom = astra.create_proj_geom('parallel', 1.0, detector_size, angles)
    
    #Object to Source (mm) = 56.135
    #Camera to Source (mm) = 225.082
    
    pixel_size = 11.32e-3*2/5
    os_distance = 56.135/pixel_size
    ds_distance = 225.082/pixel_size
    
    proj_geom = astra.create_proj_geom('fanflat', 5., detector_size, angles,
                                       os_distance, (ds_distance-os_distance))

    sinogram_id = astra.data2d.create('-sino', proj_geom, data=sinogram)
    # Create a data object for the reconstruction
    rec_id = astra.data2d.create('-vol', vol_geom)

    # Set up the parameters for a reconstruction algorithm using the GPU
    cfg = astra.astra_dict('FBP_CUDA')
    cfg['ReconstructionDataId'] = rec_id
    cfg['ProjectionDataId'] = sinogram_id
    cfg['option'] = {}
    cfg['option']['MinConstraint'] = 0
    # cfg['option']['MaxConstraint'] = 5

    # Available algorithms:
    # SIRT_CUDA, SART_CUDA, EM_CUDA, FBP_CUDA (see the FBP sample)

    # Create the algorithm object from the configuration structure
    alg_id = astra.algorithm.create(cfg)

    # Run 150 iterations of the algorithm
    astra.algorithm.run(alg_id, 1)

    # Get the result
    rec = astra.data2d.get(rec_id)

    # Clean up. Note that GPU memory is tied up in the algorithm object,
    # and main RAM in the data objects.
    astra.algorithm.delete(alg_id)
    astra.data2d.delete(rec_id)
    astra.data2d.delete(sinogram_id)
    astra.clear()
    return rec

In [ ]:
def get_sinogram():    
    def show_sinogram(i):
        with h5py.File(out_file,'r') as h5f:
            sinogram = h5f['data'][:,i,:]
        sinogram = np.array(sinogram)
        angles_reduce = 2
        size_reduce = 2
        sinogram = cv2.resize(sinogram,(sinogram.shape[1]//size_reduce, sinogram.shape[0]//angles_reduce))     
        sinogram = np.rot90(sinogram)         
        sinogram = -np.log(sinogram+1)
        sinogram = sinogram - sinogram.min()        

        plt.figure(figsize=(7,7))
        plt.imshow(sinogram, cmap=plt.cm.gray_r)
        plt.colorbar()
        plt.show()
        
        plt.figure(figsize=(10,20))
#         plt.subplot(221)
        angles = np.arange(sinogram.shape[1])*0.1*angles_reduce
#         rec_my = my_sart(sinogram, angles, [0.8,0.3,0.3])

#         print i
#         plt.imshow(rec_my, cmap=plt.cm.gray_r, vmin=0, vmax=0.05)
#         plt.title('My_sart')
        plt.subplot(211)
        angles = angles.astype('float64')/180*np.pi
        angles = angles - (angles.max()+angles.min())/2
        angles = angles + np.pi/2
        astra_data = np.rot90(sinogram)
        astra_rec = astra_tomo2d_fanflat(astra_data, angles)
        astra_rec = np.rot90(astra_rec)
        center_pos = astra_rec.shape[0]//2
        im_size = 1000
        plt.imshow(astra_rec[center_pos-im_size:center_pos+im_size,
                             center_pos-im_size:center_pos+im_size],
                   cmap=plt.cm.gray)
#         plt.colorbar(orientation='horizontal')
        plt.title('Astra')
        
        plt.subplot(212)
        source_rec = plt.imread(os.path.join(data_root,'Reconstructed','{}_rec{:04d}.png'.format(object_name, 2096-i)))
        source_rec = np.rot90(source_rec)
        source_rec = cv2.resize(source_rec,(source_rec.shape[1]//4, source_rec.shape[0]//4)) 
        plt.imshow(source_rec)
        plt.title('Skyscan')

        plt.show()
        print 'Angles count = ', sinogram.shape[1] , 'from:', angles[0], 'to:', angles[-1]-np.pi
        print 'Pixels count = ', sinogram.shape[0]
            
#     with h5py.File(out_file,'r') as h5f:
#         number_sinograms =  h5f['data'].shape[1]
        
#     interact(show_sinogram, i=(0, number_sinograms-1))
#     N=2096-960
    N=960
    interact(show_sinogram, i=(N, N+1))
    
get_sinogram()

In [ ]:
def astra_tomo2d_fanflat_interact(sinogram, angles):
    angles = angles.astype('float64')
    detector_size = sinogram.shape[1]
    

    rec_size = detector_size
    vol_geom = astra.create_vol_geom(rec_size, rec_size)
#     proj_geom = astra.create_proj_geom('parallel', 1.0, detector_size, angles)
    
    #Object to Source (mm) = 56.135
    #Camera to Source (mm) = 225.082
    
    pixel_size = 11.32e-3*2/5
    os_distance = 56.135/pixel_size
    ds_distance = 225.082/pixel_size
    
    proj_geom = astra.create_proj_geom('fanflat', 5., detector_size, angles,
                                       os_distance, (ds_distance-os_distance))

    sinogram_id = astra.data2d.create('-sino', proj_geom, data=sinogram)
    # Create a data object for the reconstruction
    rec_id = astra.data2d.create('-vol', vol_geom)

    # Set up the parameters for a reconstruction algorithm using the GPU
    cfg = astra.astra_dict('FBP_CUDA')
    cfg['ReconstructionDataId'] = rec_id
    cfg['ProjectionDataId'] = sinogram_id
    cfg['option'] = {}
    cfg['option']['MinConstraint'] = 0
    # cfg['option']['MaxConstraint'] = 5

    # Available algorithms:
    # SIRT_CUDA, SART_CUDA, EM_CUDA, FBP_CUDA (see the FBP sample)

    # Create the algorithm object from the configuration structure
    alg_id = astra.algorithm.create(cfg)

    # Run 150 iterations of the algorithm
    astra.algorithm.run(alg_id, 1)

    # Get the result
    rec = astra.data2d.get(rec_id)

    # Clean up. Note that GPU memory is tied up in the algorithm object,
    # and main RAM in the data objects.
    astra.algorithm.delete(alg_id)
    astra.data2d.delete(rec_id)
    astra.data2d.delete(sinogram_id)
    astra.clear()
    return rec

def get_sinogram():    
    def show_sinogram(i):
        with h5py.File(out_file,'r') as h5f:
            sinogram = h5f['data'][:,i,:]
        sinogram = np.array(sinogram)
        angles_reduce = 2
        size_reduce = 2
        sinogram = cv2.resize(sinogram,(sinogram.shape[1]//size_reduce, sinogram.shape[0]//angles_reduce))     
        sinogram = np.rot90(sinogram)         
        sinogram = -np.log(sinogram+1)
        sinogram = sinogram - sinogram.min()        

        
        plt.figure(figsize=(10,20))
        angles = np.arange(sinogram.shape[1])*0.1*angles_reduce
        
        plt.subplot(211)
        angles = angles.astype('float64')/180*np.pi
        angles = angles - (angles.max()+angles.min())/2
        angles = angles + np.pi/2
        astra_data = np.rot90(sinogram)
        astra_rec = astra_tomo2d_fanflat(astra_data, angles)
        astra_rec = np.rot90(astra_rec)
        center_pos = astra_rec.shape[0]//2
        im_size = 1000
        plt.imshow(astra_rec[center_pos-im_size:center_pos+im_size,
                             center_pos-im_size:center_pos+im_size],
                   cmap=plt.cm.gray)

        plt.title('Astra')
        
        plt.subplot(212)
        source_rec = plt.imread(os.path.join(data_root,'Reconstructed','{}_rec{:04d}.png'.format(object_name, 2096-i)))
        source_rec = np.rot90(source_rec)
        source_rec = cv2.resize(source_rec,(source_rec.shape[1]//4, source_rec.shape[0]//4)) 
        plt.imshow(source_rec)
        plt.title('Skyscan')

        plt.show()
        print 'Angles count = ', sinogram.shape[1] , 'from:', angles[0], 'to:', angles[-1]-np.pi
        print 'Pixels count = ', sinogram.shape[0]
            

#     N=2096-960
    N=960
    interact(show_sinogram, i=(N, N+1))
    
get_sinogram()

In [ ]:
astra.create_proj_geom??